## IPerl Notebook investigation of biomolecular interfaces 
This notebook seeks to provide the computational analysis in support of reading the classic 1994 Science paper by Spolar and Record: "Coupling of local folding to site-specific binding of proteins to DNA".

We will HackaMol and the HackaMol::X::SASA interface to the [freesasa program](http://freesasa.github.io).




In [1]:
use Modern::Perl;
use HackaMol;
use HackaMol::X::SASA;
use HackaMol::Roles::SelectionRole;
use Moose::Util qw( ensure_all_roles ); #  to apply the role to the molecule object

my $freesasa_exe = '/cluster/home/riccade/bin/freesasa';  # if this is set incorrectly, dead kernel

my $bldr = HackaMol->new; 1;

1


In order to reproduce as many of the calculations as possible (we are not attempting to match all the atomic radii nor configurations exactly), we need to be able to calculate the solvent accessible surface area, SASA, (nonpolar, polar, total) for interfaces associated with protein folding, protein-protein, and protein-DNA complexation.  

The easiest calculations to start with are protein-protein complexes using "rigid-body" association.

In Table 2.

1. Subtilisin + inhibitor -> complex        (PDBID: 2SIC)

Subtilisin is a nonspecific protease, i.e. an enzyme that digests protein. PDBID 2SIC has two chains (I for the inhibitor and E for the enzyme).  

### Notice
SASA programs generally must make some choices when calculating the surfaces of PDB protein structures. A typical PDB will contain X-tal modeled solvent water (ironically, typically not wanted for the SASA calculation) and residues with multiple configurations.  In this notebook, we will load a pdb in a two step process:
1. load the pdb into HackaMol from the PDB
2. calculate the sasa on this pdb and use the output molecule for all analysis.

Step 1. load pdb 2sic

In [2]:
my $pdb_2sic = $bldr->pdbid_mol("2sic"); 1;

1


Warning: 2sic.pdb exists, set self->overwrite(1) to overwrite at /mounts/layout/software/perlbrew/perls/p5.22.1t/lib/site_perl/5.22.1/HackaMol.pm line 26.


Step 2. run a sasa calculation and load the output molecule (that corresponds to the calculation)
* set up a scratch directory for 2SIC for the calculations

In [3]:
my $SASA_2sic    = HackaMol::X::SASA->new( scratch => "2SIC", exe=>$freesasa_exe); 1

1


* run a calculation (with default freesasa parameters)!

In [4]:
my $mol_2sic = $SASA_2sic->calc_mol($pdb_2sic);

HackaMol::Molecule=HASH(0x731de80)


Warning: overwrite mol_sasa.pdb at /mounts/layout/software/perlbrew/perls/p5.22.1t/lib/site_perl/5.22.1/HackaMol/X/SASA.pm line 300.


### let's see what we have stored in our SASA_2sic calc

In [5]:
# print $SASA_2sic->dump; # long output with pdb info.  you can do this if you want
say "Nonpolar: ", $SASA_2sic->sasa_nonpolar;
say "Polar:    ", $SASA_2sic->sasa_polar;
say "Total:    ", $SASA_2sic->sasa_total;

Nonpolar: 7909.76
Polar:    6345.73
Total:    14255.49


1


The results above are for the complex, next we need to calculate the I and E chains, isolated.
We will use a group selection and then rerun the SASA calculation on each group. 

First, let's apply the SelectionRole to mol_2sic so it is able to do selections:

In [7]:
ensure_all_roles($mol_2sic, 'HackaMol::Roles::SelectionRole');

In [10]:
$mol_2sic->clear_groups; #  just to be sure
my $ie = $mol_2sic->select_group("chain I .or. chain E");
my $i = $mol_2sic->select_group("chain I");
my $e = $mol_2sic->select_group("chain E");

$mol_2sic->push_groups($ie,$i,$e);

3


### run the calculation for each group!  

In [12]:
$SASA_2sic->calc_bygroup($mol_2sic);

Warning: overwriting stdout at reply input line 1.

overwrite mol_sasa.pdb at /mounts/layout/software/perlbrew/perls/p5.22.1t/lib/site_perl/5.22.1/HackaMol/X/SASA.pm line 300.

overwriting stdout at reply input line 1.

overwrite mol_sasa.pdb at /mounts/layout/software/perlbrew/perls/p5.22.1t/lib/site_perl/5.22.1/HackaMol/X/SASA.pm line 300.

overwriting stdout at reply input line 1.

overwrite mol_sasa.pdb at /mounts/layout/software/perlbrew/perls/p5.22.1t/lib/site_perl/5.22.1/HackaMol/X/SASA.pm line 300.


In [18]:
foreach my $igroup (0, 1, 2) {
  my $sasa_hash = $SASA_2sic->get_group_sasa($igroup);
  say "nonpolar: ", $sasa_hash->{nonpolar};
  say "polar: ",    $sasa_hash->{polar};
  say "total: ",    $sasa_hash->{total};
}

nonpolar: 7909.76
polar: 6345.73
total: 14255.49
nonpolar: 3583.16
polar: 2365.72
total: 5948.87
nonpolar: 5300.57
polar: 4631.63
total: 9932.21


### Calculations for Table 2.
$-\Delta A_{np}$, $-\Delta A_{p}$, and $-\Delta C_{p}$

where $-\Delta A_{np}$ and $-\Delta A_{p}$ are calculated directly from the differences for nonpolar and polar, respectively.  The $-\Delta C_{p}$ is calculated from Eq. 1 in the article,

$\Delta C_{p} = 0.32\Delta A_{np} - 0.14\Delta A_{p} $ (cal / mol K)

we can now calculate the change in SASA for rigid body complexation.

In [23]:
say "calculate the SASA for complexation!";
my $nonpolar_change_2sic = $SASA_2sic->get_group_sasa(0)->{nonpolar} 
                         - $SASA_2sic->get_group_sasa(1)->{nonpolar} 
                         - $SASA_2sic->get_group_sasa(2)->{nonpolar}; 
                         
my $polar_change_2sic    = $SASA_2sic->get_group_sasa(0)->{polar} 
                         - $SASA_2sic->get_group_sasa(1)->{polar} 
                         - $SASA_2sic->get_group_sasa(2)->{polar}; 
                         
say "nonpolar change:   ", $nonpolar_change_2sic;

say "polar change:      ", $polar_change_2sic;
                
say "heat capacity change:   ", 0.32*$nonpolar_change_2sic - 0.14*$polar_change_2sic;

calculate the SASA for complexation!
nonpolar change:   -973.969999999999
polar change:      -651.62
heat capacity change:   -220.4436


1
